In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

In [3]:
boom_df = pd.read_csv('boom_links.csv')
pd.options.display.max_colwidth = None
boom_df.head(30)

,url
0,https://bangla.boomlive.in/tags/puber-kolom
1,https://www.boomlive.in/tags/kathryn-robinson
2,https://www.boomlive.in/tags/jiangsu-province
3,https://www.boomlive.in/tags/Rohit-Vemula
4,https://www.boomlive.in/tags/Mufti-Raees
5,https://bangla.boomlive.in/tags/Gaiea-Sanskrit
6,https://www.boomlive.in/tags/High-School-list
7,https://www.boomlive.in/tag/factcheck/page-73
8,https://www.boomlive.in/tags/hotel-coronavirus-quarantine
9,https://www.boomlive.in/tags/Holi-in-india


In [4]:
# remove the first 20 rows

boom_df = boom_df.iloc[20: , :]
boom_df

,url
20,https://www.boomlive.in/health/does-drinking-alcohol-prevent-coronavirus-6935
21,https://www.boomlive.in/false-suicide-attempt-over-government-lic-plan
22,https://www.boomlive.in/podcast/tukde-tukde-a-jnu-story-6745
23,https://www.boomlive.in/world/can-drinking-bubble-tea-cause-gallstones-6918
24,https://www.boomlive.in/buzz/indias-case-count-rises-to-28-214
...,...
284,https://www.boomlive.in/fake-news/viral-post-claiming-96-of-rape-offenders-are-muslims-is-false-6293
285,https://hindi.boomlive.in/fake-news/photos-of-ambedkar-statue-vandalised-in-up-falsely-shared-as-delhi-7135
286,https://hindi.boomlive.in/fake-news/muslims-dominate-aaps-delhi-polls-list-party-says-not-their-list-6566
287,https://bangla.boomlive.in/fake-news/dilip-or-abhishek-who-replaced-swami-vivekananda-in-the-viral-banner-6556


In [6]:
boom_df = boom_df.reset_index(drop=True)
boom_df.head(10)

,url
0,https://www.boomlive.in/health/does-drinking-alcohol-prevent-coronavirus-6935
1,https://www.boomlive.in/false-suicide-attempt-over-government-lic-plan
2,https://www.boomlive.in/podcast/tukde-tukde-a-jnu-story-6745
3,https://www.boomlive.in/world/can-drinking-bubble-tea-cause-gallstones-6918
4,https://www.boomlive.in/buzz/indias-case-count-rises-to-28-214
5,https://www.boomlive.in/boom-reports/saudi-woman-marries-minor-son-6976
6,https://www.boomlive.in/health/false-coronavirus-found-in-broiler-chickens-6757
7,https://hindi.boomlive.in/fact-check/does-drinking-alcohol-prevent-coronavirus-6965
8,https://www.boomlive.in/fake-news/missing-child-video-wrongly-portrayed-6360
9,https://www.boomlive.in/is-this-jamia-millia-islamia-fake-female-protester


In [7]:
# remove all urls that are not fact-check articles 

boom_df = boom_df[boom_df["url"].str.contains("bangla|hindi|podcast|web-stories|budget-glossary|budget-faqs|science-technology|budget-news|buzz")==False]
boom_df

,url
0,https://www.boomlive.in/health/does-drinking-alcohol-prevent-coronavirus-6935
1,https://www.boomlive.in/false-suicide-attempt-over-government-lic-plan
3,https://www.boomlive.in/world/can-drinking-bubble-tea-cause-gallstones-6918
5,https://www.boomlive.in/boom-reports/saudi-woman-marries-minor-son-6976
6,https://www.boomlive.in/health/false-coronavirus-found-in-broiler-chickens-6757
...,...
261,https://www.boomlive.in/world/coronavirus/australian-news-report-edited-to-show-bananas-defend-against-covid-19-7266
262,https://www.boomlive.in/fake-news/did-pm-modi-call-himself-a-pathans-son-the-real-context-6901
263,https://www.boomlive.in/coronavirus-outbreak/dr-devi-shetty-did-not-record-viral-audio-clip-on-coronavirus-7275
264,https://www.boomlive.in/fake-news/viral-post-claiming-96-of-rape-offenders-are-muslims-is-false-6293


In [8]:
boom_df = boom_df.reset_index(drop=True)
boom_df

,url
0,https://www.boomlive.in/health/does-drinking-alcohol-prevent-coronavirus-6935
1,https://www.boomlive.in/false-suicide-attempt-over-government-lic-plan
2,https://www.boomlive.in/world/can-drinking-bubble-tea-cause-gallstones-6918
3,https://www.boomlive.in/boom-reports/saudi-woman-marries-minor-son-6976
4,https://www.boomlive.in/health/false-coronavirus-found-in-broiler-chickens-6757
...,...
202,https://www.boomlive.in/world/coronavirus/australian-news-report-edited-to-show-bananas-defend-against-covid-19-7266
203,https://www.boomlive.in/fake-news/did-pm-modi-call-himself-a-pathans-son-the-real-context-6901
204,https://www.boomlive.in/coronavirus-outbreak/dr-devi-shetty-did-not-record-viral-audio-clip-on-coronavirus-7275
205,https://www.boomlive.in/fake-news/viral-post-claiming-96-of-rape-offenders-are-muslims-is-false-6293


In [9]:
boom_df.to_csv("boom_try2.csv", index=False)

In [10]:
boomlive = pd.read_csv('boom_try2.csv')
pd.options.display.max_colwidth = None
boomlive.head()

,url
0,https://www.boomlive.in/health/does-drinking-alcohol-prevent-coronavirus-6935
1,https://www.boomlive.in/false-suicide-attempt-over-government-lic-plan
2,https://www.boomlive.in/world/can-drinking-bubble-tea-cause-gallstones-6918
3,https://www.boomlive.in/boom-reports/saudi-woman-marries-minor-son-6976
4,https://www.boomlive.in/health/false-coronavirus-found-in-broiler-chickens-6757


In [11]:
boomlive.shape

(207, 1)

In [12]:
boomlive_df = boomlive.url.tolist()
boomlive_df

['https://www.boomlive.in/health/does-drinking-alcohol-prevent-coronavirus-6935',
 'https://www.boomlive.in/false-suicide-attempt-over-government-lic-plan',
 'https://www.boomlive.in/world/can-drinking-bubble-tea-cause-gallstones-6918',
 'https://www.boomlive.in/boom-reports/saudi-woman-marries-minor-son-6976',
 'https://www.boomlive.in/health/false-coronavirus-found-in-broiler-chickens-6757',
 'https://www.boomlive.in/fake-news/missing-child-video-wrongly-portrayed-6360',
 'https://www.boomlive.in/is-this-jamia-millia-islamia-fake-female-protester',
 'https://www.boomlive.in/fact-file/can-cops-enter-university-campuses-6348',
 'https://www.boomlive.in/boom-reports/sundar-pichai-commented-on-caa-6410',
 'https://www.boomlive.in/health/viral-coronavirus-advisory-is-not-from-unicef-7171',
 'https://www.boomlive.in/man-thrashed-for-lewd-behaviour,-communal-spin-viral',
 'https://www.boomlive.in/fake-news/chhatrapati-shivaji-maharaj-on-100-bill-6589',
 'https://www.boomlive.in/factcheck/di

In [15]:
boom_list = []
for url in tqdm(boomlive_df):
    boom_dict = {}
    response = requests.get(url)
    doc = BeautifulSoup(response.content, "html.parser")
    
    title = doc.find(class_='entry-title mb-10')
    boom_dict['headline'] = title.string
    
    date = doc.find(class_='convert-to-localtime')
    boom_dict['datetime'] = date.string
    
    boom_list.append(boom_dict)

boom_list  

100%|█████████████████████████████████████████| 207/207 [12:36<00:00,  3.66s/it]


[{'headline': 'Does Drinking Alcohol Prevent Coronavirus?',
  'datetime': '19 Feb 2020 1:15 PM GMT'},
 {'headline': "False: Suicide Attempt Over Government's LIC Stake Sale Plan",
  'datetime': '3 Feb 2020 12:33 PM GMT'},
 {'headline': 'Can Drinking Bubble Tea Cause Gallstones?',
  'datetime': '17 Feb 2020 1:49 PM GMT'},
 {'headline': 'Saudi Woman Marries Minor Son?',
  'datetime': '24 Feb 2020 5:44 AM GMT'},
 {'headline': 'False: Coronavirus Found In Broiler Chickens',
  'datetime': '4 Feb 2020 10:10 AM GMT'},
 {'headline': 'Missing Child Video Wrongly Portrayed',
  'datetime': '20 Dec 2019 2:01 PM GMT'},
 {'headline': "Is This Jamia Millia Islamia's 'Fake' Female Protester?",
  'datetime': '16 Dec 2019 6:20 PM GMT'},
 {'headline': 'Can Cops Enter University Campuses?',
  'datetime': '19 Dec 2019 10:05 AM GMT'},
 {'headline': 'Sundar Pichai Commented On CAA?',
  'datetime': '26 Dec 2019 3:11 PM GMT'},
 {'headline': "Viral Coronavirus 'Advisory' Is Not From UNICEF",
  'datetime': '9 Ma

In [16]:
len(boom_list)

207

In [17]:
boom_new_df = pd.DataFrame(boom_list)
boom_new_df

,headline,datetime
0,Does Drinking Alcohol Prevent Coronavirus?,19 Feb 2020 1:15 PM GMT
1,False: Suicide Attempt Over Government's LIC Stake Sale Plan,3 Feb 2020 12:33 PM GMT
2,Can Drinking Bubble Tea Cause Gallstones?,17 Feb 2020 1:49 PM GMT
3,Saudi Woman Marries Minor Son?,24 Feb 2020 5:44 AM GMT
4,False: Coronavirus Found In Broiler Chickens,4 Feb 2020 10:10 AM GMT
...,...,...
202,Australian News Report Edited To Show Bananas Defend Against COVID-19,18 March 2020 12:23 PM GMT
203,Did PM Modi Call Himself 'A Pathan's Son'? The Real Context,15 Feb 2020 12:21 PM GMT
204,Dr. Devi Shetty Did Not Record Viral Audio Clip On Coronavirus,19 March 2020 8:52 AM GMT
205,Viral Post Claiming 96% Of Rape Offenders Are Muslims Is False,14 Dec 2019 7:01 AM GMT


In [19]:
boom_live_df2 = pd.concat([boomlive, boom_new_df], axis=1, join='inner')

In [20]:
boom_live_df2

,url,headline,datetime
0,https://www.boomlive.in/health/does-drinking-alcohol-prevent-coronavirus-6935,Does Drinking Alcohol Prevent Coronavirus?,19 Feb 2020 1:15 PM GMT
1,https://www.boomlive.in/false-suicide-attempt-over-government-lic-plan,False: Suicide Attempt Over Government's LIC Stake Sale Plan,3 Feb 2020 12:33 PM GMT
2,https://www.boomlive.in/world/can-drinking-bubble-tea-cause-gallstones-6918,Can Drinking Bubble Tea Cause Gallstones?,17 Feb 2020 1:49 PM GMT
3,https://www.boomlive.in/boom-reports/saudi-woman-marries-minor-son-6976,Saudi Woman Marries Minor Son?,24 Feb 2020 5:44 AM GMT
4,https://www.boomlive.in/health/false-coronavirus-found-in-broiler-chickens-6757,False: Coronavirus Found In Broiler Chickens,4 Feb 2020 10:10 AM GMT
...,...,...,...
202,https://www.boomlive.in/world/coronavirus/australian-news-report-edited-to-show-bananas-defend-against-covid-19-7266,Australian News Report Edited To Show Bananas Defend Against COVID-19,18 March 2020 12:23 PM GMT
203,https://www.boomlive.in/fake-news/did-pm-modi-call-himself-a-pathans-son-the-real-context-6901,Did PM Modi Call Himself 'A Pathan's Son'? The Real Context,15 Feb 2020 12:21 PM GMT
204,https://www.boomlive.in/coronavirus-outbreak/dr-devi-shetty-did-not-record-viral-audio-clip-on-coronavirus-7275,Dr. Devi Shetty Did Not Record Viral Audio Clip On Coronavirus,19 March 2020 8:52 AM GMT
205,https://www.boomlive.in/fake-news/viral-post-claiming-96-of-rape-offenders-are-muslims-is-false-6293,Viral Post Claiming 96% Of Rape Offenders Are Muslims Is False,14 Dec 2019 7:01 AM GMT


In [21]:
boom_live_df2['organization'] = "Boomlive"
boom_live_df2

,url,headline,datetime,organization
0,https://www.boomlive.in/health/does-drinking-alcohol-prevent-coronavirus-6935,Does Drinking Alcohol Prevent Coronavirus?,19 Feb 2020 1:15 PM GMT,Boomlive
1,https://www.boomlive.in/false-suicide-attempt-over-government-lic-plan,False: Suicide Attempt Over Government's LIC Stake Sale Plan,3 Feb 2020 12:33 PM GMT,Boomlive
2,https://www.boomlive.in/world/can-drinking-bubble-tea-cause-gallstones-6918,Can Drinking Bubble Tea Cause Gallstones?,17 Feb 2020 1:49 PM GMT,Boomlive
3,https://www.boomlive.in/boom-reports/saudi-woman-marries-minor-son-6976,Saudi Woman Marries Minor Son?,24 Feb 2020 5:44 AM GMT,Boomlive
4,https://www.boomlive.in/health/false-coronavirus-found-in-broiler-chickens-6757,False: Coronavirus Found In Broiler Chickens,4 Feb 2020 10:10 AM GMT,Boomlive
...,...,...,...,...
202,https://www.boomlive.in/world/coronavirus/australian-news-report-edited-to-show-bananas-defend-against-covid-19-7266,Australian News Report Edited To Show Bananas Defend Against COVID-19,18 March 2020 12:23 PM GMT,Boomlive
203,https://www.boomlive.in/fake-news/did-pm-modi-call-himself-a-pathans-son-the-real-context-6901,Did PM Modi Call Himself 'A Pathan's Son'? The Real Context,15 Feb 2020 12:21 PM GMT,Boomlive
204,https://www.boomlive.in/coronavirus-outbreak/dr-devi-shetty-did-not-record-viral-audio-clip-on-coronavirus-7275,Dr. Devi Shetty Did Not Record Viral Audio Clip On Coronavirus,19 March 2020 8:52 AM GMT,Boomlive
205,https://www.boomlive.in/fake-news/viral-post-claiming-96-of-rape-offenders-are-muslims-is-false-6293,Viral Post Claiming 96% Of Rape Offenders Are Muslims Is False,14 Dec 2019 7:01 AM GMT,Boomlive


In [22]:
# remove the time from the datetime column

boom_live_df2['datetime'] = boom_live_df2['datetime'].str.replace(r'\d+:\d{2}\s\w{2}\s\w{3}$', '')
boom_live_df2

/var/folders/3x/_wb_w3ys63j7vqmgfcp3xx_w0000gn/T/ipykernel_2380/1402353588.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  boom_live_df2['datetime'] = boom_live_df2['datetime'].str.replace(r'\d+:\d{2}\s\w{2}\s\w{3}$', '')


,url,headline,datetime,organization
0,https://www.boomlive.in/health/does-drinking-alcohol-prevent-coronavirus-6935,Does Drinking Alcohol Prevent Coronavirus?,19 Feb 2020,Boomlive
1,https://www.boomlive.in/false-suicide-attempt-over-government-lic-plan,False: Suicide Attempt Over Government's LIC Stake Sale Plan,3 Feb 2020,Boomlive
2,https://www.boomlive.in/world/can-drinking-bubble-tea-cause-gallstones-6918,Can Drinking Bubble Tea Cause Gallstones?,17 Feb 2020,Boomlive
3,https://www.boomlive.in/boom-reports/saudi-woman-marries-minor-son-6976,Saudi Woman Marries Minor Son?,24 Feb 2020,Boomlive
4,https://www.boomlive.in/health/false-coronavirus-found-in-broiler-chickens-6757,False: Coronavirus Found In Broiler Chickens,4 Feb 2020,Boomlive
...,...,...,...,...
202,https://www.boomlive.in/world/coronavirus/australian-news-report-edited-to-show-bananas-defend-against-covid-19-7266,Australian News Report Edited To Show Bananas Defend Against COVID-19,18 March 2020,Boomlive
203,https://www.boomlive.in/fake-news/did-pm-modi-call-himself-a-pathans-son-the-real-context-6901,Did PM Modi Call Himself 'A Pathan's Son'? The Real Context,15 Feb 2020,Boomlive
204,https://www.boomlive.in/coronavirus-outbreak/dr-devi-shetty-did-not-record-viral-audio-clip-on-coronavirus-7275,Dr. Devi Shetty Did Not Record Viral Audio Clip On Coronavirus,19 March 2020,Boomlive
205,https://www.boomlive.in/fake-news/viral-post-claiming-96-of-rape-offenders-are-muslims-is-false-6293,Viral Post Claiming 96% Of Rape Offenders Are Muslims Is False,14 Dec 2019,Boomlive


In [23]:
boom_live_df2['datetime'] = pd.to_datetime(boom_live_df2.datetime)

In [24]:
boom_live_df2.dtypes

url                     object
headline                object
datetime        datetime64[ns]
organization            object
dtype: object

In [26]:
boom_live_df2

,url,headline,datetime,organization
0,https://www.boomlive.in/health/does-drinking-alcohol-prevent-coronavirus-6935,Does Drinking Alcohol Prevent Coronavirus?,2020-02-19,Boomlive
1,https://www.boomlive.in/false-suicide-attempt-over-government-lic-plan,False: Suicide Attempt Over Government's LIC Stake Sale Plan,2020-02-03,Boomlive
2,https://www.boomlive.in/world/can-drinking-bubble-tea-cause-gallstones-6918,Can Drinking Bubble Tea Cause Gallstones?,2020-02-17,Boomlive
3,https://www.boomlive.in/boom-reports/saudi-woman-marries-minor-son-6976,Saudi Woman Marries Minor Son?,2020-02-24,Boomlive
4,https://www.boomlive.in/health/false-coronavirus-found-in-broiler-chickens-6757,False: Coronavirus Found In Broiler Chickens,2020-02-04,Boomlive
...,...,...,...,...
202,https://www.boomlive.in/world/coronavirus/australian-news-report-edited-to-show-bananas-defend-against-covid-19-7266,Australian News Report Edited To Show Bananas Defend Against COVID-19,2020-03-18,Boomlive
203,https://www.boomlive.in/fake-news/did-pm-modi-call-himself-a-pathans-son-the-real-context-6901,Did PM Modi Call Himself 'A Pathan's Son'? The Real Context,2020-02-15,Boomlive
204,https://www.boomlive.in/coronavirus-outbreak/dr-devi-shetty-did-not-record-viral-audio-clip-on-coronavirus-7275,Dr. Devi Shetty Did Not Record Viral Audio Clip On Coronavirus,2020-03-19,Boomlive
205,https://www.boomlive.in/fake-news/viral-post-claiming-96-of-rape-offenders-are-muslims-is-false-6293,Viral Post Claiming 96% Of Rape Offenders Are Muslims Is False,2019-12-14,Boomlive


In [25]:
boom_live_df2.to_csv('boom_df2.csv', index=False)

#### Note: The csv file has been renamed to boom_df.
#### This is the second scraping and cleaning. The first one, I realised, did not pull all the needed articles.
#### So I've deleted the first notebook and the first df and renamed the csv.